In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [6]:
from langchain_groq import ChatGroq

groq_api_key= os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]= os.getenv("HF_TOKEN")

In [7]:
llm= ChatGroq(groq_api_key= groq_api_key, model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026A5E17FF50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026A5E223DD0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings= HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\My_Work\UdeMy\MyGenAI\Chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## VectorStores

In [10]:
from langchain_chroma import Chroma

vectorstore= Chroma.from_documents(documents, embedding=embeddings)
vectorstore

In [12]:
vectorstore.similarity_search("goldfish")

[Document(id='14f6fba5-d5dc-447a-9aa7-64f6d2bc090b', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(id='e3a81bb7-f82b-4659-a468-fa3d07118ed6', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(id='75a018e4-3e14-4b65-8987-e6083640286a', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='8761cd73-3cd3-40cc-8c6d-d812560e9f96', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [13]:
vectorstore.similarity_search("cat")

[Document(id='8761cd73-3cd3-40cc-8c6d-d812560e9f96', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='75a018e4-3e14-4b65-8987-e6083640286a', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='7db85321-49b3-43ec-8f1e-af4d8a736a7a', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='e3a81bb7-f82b-4659-a468-fa3d07118ed6', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [16]:
## Async Query

await vectorstore.asimilarity_search("cat")

[Document(id='8761cd73-3cd3-40cc-8c6d-d812560e9f96', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='75a018e4-3e14-4b65-8987-e6083640286a', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='7db85321-49b3-43ec-8f1e-af4d8a736a7a', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='e3a81bb7-f82b-4659-a468-fa3d07118ed6', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [17]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='8761cd73-3cd3-40cc-8c6d-d812560e9f96', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(id='75a018e4-3e14-4b65-8987-e6083640286a', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.574089765548706),
 (Document(id='7db85321-49b3-43ec-8f1e-af4d8a736a7a', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(id='e3a81bb7-f82b-4659-a468-fa3d07118ed6', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.665792465209961)]

### Retrievers
LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

In [18]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever= RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='8761cd73-3cd3-40cc-8c6d-d812560e9f96', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='75a018e4-3e14-4b65-8987-e6083640286a', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

#### Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever.

In [19]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])


[[Document(id='8761cd73-3cd3-40cc-8c6d-d812560e9f96', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='75a018e4-3e14-4b65-8987-e6083640286a', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

## RAG

In [20]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain= {"context":retriever,"question":RunnablePassthrough()}|prompt|llm

response= rag_chain.invoke("tell me about dogs")

print(response.content)

According to the provided context, dogs are great companions, known for their loyalty and friendliness.
